In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

: 

### Pegando Links dos temas
Pagina: https://educacao.uol.com.br/bancoderedacoes/

In [71]:
html = open('main_page.txt','r').read()
main_page = BeautifulSoup(html)
temas = main_page.find_all('div', attrs={'class':'thumbnails-item'}) # divs
temas_links = [link.find('a').get('href') for link in temas] # hrefs
print(f'Encontrados {len(temas_links)} links de diferentes temas do ENEM:\nExemplo: {temas_links[:2]}')

Encontrados 55 links de diferentes temas do ENEM:
Exemplo: ['https://educacao.uol.com.br/bancoderedacoes/propostas/qualificacao-e-o-futuro-do-emprego.htm', 'https://educacao.uol.com.br/bancoderedacoes/propostas/supremo-tribunal-federal-e-opiniao-publica.htm']


### Seguindo Links

In [72]:
# faz um request para cada link de tema
temas_paginas = []
for link in temas_links:
    pagina_do_tema = requests.get(link).text
    pagina_do_tema = BeautifulSoup(pagina_do_tema)
    temas_paginas.append(pagina_do_tema)

In [128]:
banco = {} # banco de redacoes
qtd_redacoes = 0
for pagina in temas_paginas:
    redacoes = pagina.find_all('section',attrs={'class':'results-table'})
    # deve retornar um unico objeto HTML
    if len(redacoes) != 1:
        # print(f'Tema {nome_do_tema} não possui redacoes')
        continue
    #
    nome_do_tema = pagina.find_all('i', attrs={'custom-title'})[0].get_text().strip()
    redacoes = redacoes[0].find_all('div', attrs={'class':'rt-line-option'})
    redacoes_links = [red.find_all('a')[0].get('href') for red in redacoes]
    banco[nome_do_tema] = redacoes_links
    qtd_redacoes += len(redacoes_links)
    # print(f'Para o tema "{nome_do_tema}" foram encontradas {len(redacoes_links)} redacoes corrigidas') 
print(f'Total de {len(banco.keys())} temas com {qtd_redacoes} redacoes(links)')

Total de 42 temas com 834 redacoes(links)


### Acessando Redações

In [ ]:

        red[''] = ''
        red[''] = ''
        red[''] = ''
        red[''] = ''

In [184]:
MAPPING_TOPICS = [
    'Demonstrar domínio da norma culta da língua escrita.',
    'Compreender a proposta da redação e aplicar conceito das várias áreas de conhecimento para desenvolver o tema, dentro dos limites estruturais do texto dissertativo-argumentativo.',
    'Selecionar, relacionar, organizar e interpretar informações, fatos, opiniões e argumentos em defesa de um ponto de vista.',
    'Demonstrar conhecimento dos mecanismos linguísticos necessários para a construção da argumentação.',
    'Elaborar a proposta de solução para o problema abordado, mostrando respeito aos valores humanos e considerando a diversidade sociocultural.',
    'Nota final']

In [195]:
redacoes_completas = []

for tema, redacoes_url in banco.items():
    for url in redacoes_url[:2]:
        resp = BeautifulSoup(requests.get(url).text)
        # cria objeto
        red = {}
        red['tema'] = tema
        red['titulo'] = resp.find_all('h2')[-1].get_text().strip()
        red['nota_final'] = resp.find_all('span', attrs={'class':'mark'})[0].get_text().strip()
        #
        #
        #
        # tabela com nota para as competencias:
        competencias_table = resp.find_all('section',attrs={'class':'results-table'})[0]
        topics = competencias_table.find_all('span',attrs={'class':'topic'})
        topics = [t.get_text() for t in topics]
        # notas
        points = competencias_table.find_all('span',attrs={'class':'points'})
        points = [t.get_text() for t in points]
        points
        competencias_com_nota = {k[0]:k[1] for k in list(zip(topics, points))}
        
        red['comp_lingua_culta'] = competencias_com_nota[MAPPING_TOPICS[0]]
        red['comp_proposta'] = competencias_com_nota[MAPPING_TOPICS[1]]
        red['comp_argumentacao'] = competencias_com_nota[MAPPING_TOPICS[2]]
        red['comp_conhecimentos'] = competencias_com_nota[MAPPING_TOPICS[3]]
        red['comp_proposta_solucao'] = competencias_com_nota[MAPPING_TOPICS[4]]
        #
        #
        #
        red['url'] = url
        red['texto_original'] = ''
        red['texto_com_anotacoes'] = ''
        red['comentario_geral'] = ''
        red['competencias_resumo'] = ''
        #
        #
        #
        redacoes_completas.append(red)
    break

In [196]:
red

{'tema': 'Qualificação e o futuro do emprego',
 'titulo': 'Qualificação será o futuro do emprego',
 'nota_final': '220',
 'comp_lingua_culta': '80',
 'comp_proposta': '80',
 'comp_argumentacao': '20',
 'comp_conhecimentos': '20',
 'comp_proposta_solucao': '20',
 'url': 'https://educacao.uol.com.br/bancoderedacoes/redacoes/qualificacao-sera-o-futuro-do-emprego.htm',
 'texto_original': '',
 'texto_com_anotacoes': '',
 'comentario_geral': '',
 'competencias_resumo': ''}

In [192]:
redacoes_completas_df = pd.DataFrame(redacoes_completas)